This is a cell of text! (Markdown cell)

It does nothing. 

In [236]:
import numpy as np, math, matplotlib as plot

In [344]:
# in this box we're going to define our grid and data

'''this will be size of grid'''
gridSize = 100 
testGridSize = 9

# produce random data values 
'''eventually this will be brightness temperature data'''
data0 = np.random.rand(np.sqrt(gridSize),np.sqrt(gridSize)) * 10
# copy the original data and artificially move the array to the right
data1 = data0.copy()
data1 = np.hstack((data1[:,-1].reshape((data1.shape[0], 1)),data1[:,0:-1]))

# define indices
I, J = np.meshgrid(np.arange(0,int(np.sqrt(gridSize))),np.arange(0,int(np.sqrt(gridSize))),indexing='ij')
I = I.reshape((I.size, 1))
J = J.reshape((J.size, 1))

# define locations
'''eventually this will be lat and lon data'''
Y, X = np.meshgrid(10*np.arange(0,int(np.sqrt(gridSize))),10*np.arange(0,int(np.sqrt(gridSize))),indexing='ij')
XkeepShape = X
YkeepShape = Y
X = X.reshape((X.size, 1))
Y = Y.reshape((Y.size, 1))

N = np.arange(0,X.size)
NkeepShape = N.copy()
NkeepShape = NkeepShape.reshape((int(np.sqrt(gridSize)), int(np.sqrt(gridSize))))
N = N.reshape((N.size, 1))

# keep all location data in one array
locationData = np.concatenate((N,I,J,Y,X),1)

# define a don't bother with mask
''' this will contain: landmask (and half landmask to avoid footprint overlap), also mask any point with bad data'''
dontBotherMask = np.zeros_like(data0).flatten()

# produce an index for transforming data to list of (overlapping) testGrids
transformSubI = np.arange(-int(np.floor(np.sqrt(testGridSize)/2)),int(np.floor(np.sqrt(testGridSize)/2))+1,1)
transformSubI = np.tile(transformSubI.reshape(transformSubI.size, 1),(1, int(np.sqrt(testGridSize))))
transformSubI = transformSubI.reshape((1, transformSubI.size))
#print transformSubI

# produce an index for transforming data to list of (overlapping) testGrids
transformSubJ = np.arange(-int(np.floor(np.sqrt(testGridSize)/2)),int(np.floor(np.sqrt(testGridSize)/2))+1,1)
transformSubJ = np.tile(transformSubJ.reshape(1, transformSubJ.size),(int(np.sqrt(testGridSize)), 1))
transformSubJ = transformSubJ.reshape((1, transformSubJ.size))
#print transformSubJ

# broadcast the transformSubs with their big counterparts
transformSubI = transformSubI + I
transformSubJ = transformSubJ + J

#now remove the test grids if they:
#    1) contain any negative numbers in I or J
#    2) if the dontBotherMask is set
removeData = np.logical_or(np.logical_or(transformSubI<0,transformSubJ<0),
                           np.logical_or(transformSubI>=data1.shape[0],transformSubJ>=data1.shape[1]))
removeData = np.logical_or(np.any(removeData,axis=1),dontBotherMask)

transformSubI = transformSubI[removeData==0,:]
transformSubJ = transformSubJ[removeData==0,:]
locationData = locationData[removeData==0,:]
# make an RDD of the index and location data, and also a broadcast variable
locationBroadcast = sc.broadcast(locationData)

# transform the data:
data0 = data0[transformSubI,transformSubJ]
data1 = data1[transformSubI,transformSubJ]

print removeData
print type(removeData)

[ True  True  True  True  True  True  True  True  True  True  True False
 False False False False False False False  True  True False False False
 False False False False False  True  True False False False False False
 False False False  True  True False False False False False False False
 False  True  True False False False False False False False False  True
  True False False False False False False False False  True  True False
 False False False False False False False  True  True False False False
 False False False False False  True  True  True  True  True  True  True
  True  True  True  True]
<type 'numpy.ndarray'>


In [379]:
NBroadcast = sc.broadcast(NkeepShape)
YBroadcast = sc.broadcast(YkeepShape)
XBroadcast = sc.broadcast(XkeepShape)

#define a function which returns the Euclidean distance between two points (in the x, y plane)
def distance_calc(x1,y1,x2,y2):
        
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    
    return distance

def distance_calcRDD(line):
            
    Xs = XBroadcast.value[line[1],line[2]]
    Ys = YBroadcast.value[line[1],line[2]]
    x1 = Xs[np.floor(float(Xs.size)/2)]
    y1 = Ys[np.floor(float(Ys.size)/2)]
    
    distance = distance_calc(x1,y1,Xs,Ys)
    Ns = NBroadcast.value[line[1],line[2]]
    
    keepData = np.logical_and(np.logical_and(distance<=distThresholdBC.value,distance!=0),
                          np.any(Ns==locationBroadcast.value[:,0:1],axis=0))
    
    Ns = Ns[keepData==1]
    
    return (line[0], list(Ns))
    
    
# Doing a cartesian product of every grid cell against every other grid cell is really, really inefficient.
# So, first we will filter the combinations of matches by their distance between each other on the grid.
# distGridThreshold is the maximum number of grid spaces which can be moved
'''eventually this will be: max(possible distance to move)/min(possible distances between grid points)'''
distGridThreshold = 4

# this is the physical distance that will be the threshold for the movement of ice
'''eventually will be the distance possible to move (in km) over 1 day at 1m/s (24 * 60 * 60 * 1 / 1000)'''
distThresholdBC = sc.broadcast(30)

# produce an index for checking distances
distCheckSubI = np.arange(-distGridThreshold,distGridThreshold+1,1)
distCheckSubI = np.tile(distCheckSubI.reshape(distCheckSubI.size, 1),(1, 2*distGridThreshold+1))
distCheckSubI = distCheckSubI.reshape((1, distCheckSubI.size))
distCheckSubI = distCheckSubI + I
distCheckSubI = distCheckSubI[removeData==0,:]

# produce an index for checking distances
distCheckSubJ = np.arange(-distGridThreshold,distGridThreshold+1,1)
distCheckSubJ = np.tile(distCheckSubJ.reshape(1, distCheckSubJ.size),(2*distGridThreshold+1, 1))
distCheckSubJ = distCheckSubJ.reshape((1, distCheckSubJ.size))
distCheckSubJ = distCheckSubJ + J
distCheckSubJ = distCheckSubJ[removeData==0,:]

# replace any data which is out of bounds with [0, 0]
badData = np.logical_or(np.logical_or(distCheckSubI<0,distCheckSubI>=XkeepShape.shape[0]),
           np.logical_or(distCheckSubJ<0,distCheckSubJ>=XkeepShape.shape[1]))

distCheckSubI[badData==1] = 0
distCheckSubJ[badData==1] = 0

# chuck the location number (which is ABSOLUTE, rather than tied to locationData size) and the indices
# into an RDD and map to a len=3 tuple
distCheckSub = np.concatenate((locationData[:,0:1],distCheckSubI,distCheckSubJ),1)
distCheckSubRDD = (sc.parallelize(distCheckSub,4)
                   .map(lambda a: (a[0],a[1:(2*distGridThreshold+1)**2+1], a[(2*distGridThreshold+1)**2+1:])))

distanceDictRDD = (distCheckSubRDD
                   .map(distance_calcRDD)
                   .cache())

#print distanceDictRDD.filter(lambda (a, b): a==31).collect()
#print distanceDictRDD.collect()





In [367]:


#reformat the distanceDictRDD to the correct dictionary-like object
def makeDistFunc(line):
    
    key = line[0]
    tempList = line[1]
    
    return (key, list(line[1]))


#locationDataRDD = sc.parallelize(locationData, 4)

# take the cartesian product and compute the distance between each point: filter by distance threshold
# distanceDataRDD contains the info about each point, and the other points which are less than the distance
# threshold away
#'''this process needs to be sped up'''
#distanceDataRDD = (locationDataRDD
#                   .cartesian(locationDataRDD)
#                   .map(lambda (a, b): (a, (b, distance_calc(a[3],a[4],b[3],b[4]))))
#                   .filter(lambda (a, (b, dist)): dist < distThreshold and dist != 0)
#                   .map(lambda (a, (b, dist)): (a[0], b[0])))

# now reduce by key to list those points within range
distanceDictRDD = (distanceDataRDD
                   .groupByKey()
                   .map(makeDistFunc)
                   .sortByKey()
                   .cache())

print distanceDictRDD.filter(lambda (a, b): a==16).collect()
#print np.arange(0,100).reshape((10,10))
#print distanceDictRDD.take(20)

[(16, [14, 15, 17, 18, 24, 25, 26, 27, 28, 34, 35, 36, 37, 38])]


In [380]:
# define a function which collects the broadcast data into an RDD
# can also put correlation in here if you like
def computeCorrelation(line):
        
    # extract relevant data from the daily broadcast variables
    getdata0 = data0Broadcast.value[line[0]==locationBroadcast.value[:,0],:]
    
    grabBinary = np.any(locationBroadcast.value[:,0].reshape((locationBroadcast.value[:,0].size, 1))==line[1],axis=1)
    getdata1 = data1Broadcast.value[grabBinary,:]
            
    # tile data0 for fast correlation
    getdata0 = np.tile(getdata0-np.mean(getdata0),(getdata1.shape[0], 1))
    getdata1 = getdata1 - np.mean(getdata1,axis=1).reshape((getdata1.shape[0], 1))
    
    correl = np.sum(getdata0*getdata1,axis=1) / (np.sqrt(np.sum(getdata0*getdata0,axis=1)) * np.sqrt(np.sum(getdata1*getdata1,axis=1)))
    
    return (line[0], line[1], correl)

# find the largest correlation
'''this also needs work on: threshold value, and sharpness of peak.
Can find threshold value by computing distribution of correlations amongst motions which could never occur.
Then take 90th percentile: should remove all but 10% of bad matches.'''
def findLargestCorrel(line):
    
    correl = line[2]
    maximum = line[1][correl.argmax()]
    
    xvel = locationBroadcast.value[maximum==locationBroadcast.value[:,0],4] - locationBroadcast.value[line[0]==locationBroadcast.value[:,0],4]
    yvel = locationBroadcast.value[maximum==locationBroadcast.value[:,0],3] - locationBroadcast.value[line[0]==locationBroadcast.value[:,0],3]
        
    i = locationBroadcast.value[line[0]==locationBroadcast.value[:,0],1]
    j = locationBroadcast.value[line[0]==locationBroadcast.value[:,0],2]
    
    return (np.hstack((i, j)), np.hstack((yvel,xvel)))

print findLargestCorrel(computeCorrelation((31, [11, 12, 13, 21, 22, 23, 32, 33, 41, 42, 43, 51, 52, 53])))

(array([3, 1]), array([ 0, 10]))


In [381]:
# turn data0 and data1 into broadcast variables
data0Broadcast = sc.broadcast(data0)
data1Broadcast = sc.broadcast(data1)

correlationRDD = (distanceDictRDD
            .map(computeCorrelation)
            .map(findLargestCorrel)
            .cache())
            
#print correlationRDD.collect()
#print np.arange(0,100).reshape((10,10))

In [382]:
# now need to reconstruct original data format
# to do this, we are going to collect 2 RDDS

# one for the location of output
outputLocation = np.array(correlationRDD
                 .flatMap(lambda (a, b): a)
                 .collect())
outputLocation = outputLocation.reshape((outputLocation.size/2, 2))

# one for the distance travelled
outputDist = np.array(correlationRDD
                 .flatMap(lambda (a, b): b)
                 .collect())
outputDist = outputDist.reshape((outputDist.size/2, 2))


yDist = np.ones((np.sqrt(gridSize),np.sqrt(gridSize)))*np.nan
xDist = np.ones((np.sqrt(gridSize),np.sqrt(gridSize)))*np.nan

'''These need to be divided by the time period'''
yDist[outputLocation[:,0],outputLocation[:,1]] = outputDist[:,0]
xDist[outputLocation[:,0],outputLocation[:,1]] = outputDist[:,1]

#print outputLocation
#print outputLocation[1:2:]
print yDist
print xDist



[[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan   0.   0.   0.   0.   0.   0.   0.  20.  nan]
 [ nan   0.   0.   0.   0.   0.   0.   0.  10.  nan]
 [ nan   0.   0.   0.   0.   0.   0.   0. -20.  nan]
 [ nan   0.   0.   0.   0.   0.   0.   0. -20.  nan]
 [ nan   0.   0.   0.   0.   0.   0.   0. -20.  nan]
 [ nan   0.   0.   0.   0.   0.   0.   0. -30.  nan]
 [ nan   0.   0.   0.   0.   0.   0.   0. -30.  nan]
 [ nan   0.   0.   0.   0.   0.   0.   0. -10.  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]]
[[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  10.  10.  10.  10.  10.  10.  10. -20.  nan]
 [ nan  10.  10.  10.  10.  10.  10.  10. -10.  nan]
 [ nan  10.  10.  10.  10.  10.  10.  10. -10.  nan]
 [ nan  10.  10.  10.  10.  10.  10.  10. -10.  nan]
 [ nan  10.  10.  10.  10.  10.  10.  10. -10.  nan]
 [ nan  10.  10.  10.  10.  10.  10.  10.   0.  nan]
 [ nan  10.  10.  10.  10.  10.  10.  10.   0.  nan]
 [ nan  10.  10.  10.  10.  10.  10.  10. -20